In [1]:
import numpy as np
import scipy.io as sio
from glob import glob
from tqdm import tqdm
from scipy.signal import convolve2d
from scipy.interpolate import RectBivariateSpline
from astropy.modeling.blackbody import blackbody_lambda, blackbody_nu
from astropy.modeling import models, fitting
import astropy.units as u
import pickle
import matplotlib.pyplot as plt
from astropy.io import fits
import sys
sys.path.append('../python/')
import chroma
%matplotlib inline
psfdir='../psf_models/psf_data_8_18_08/PSF_Mono/'

In [ ]:
module=2
output=1
fnames=glob('{}Module_{}/{}/*.mat'.format(psfdir,module,module+0.1*output))
lam=np.asarray([(f.split('_')[-1])[0:-4] for f in fnames],dtype='float')
psf_model=np.zeros((24,4,len(fnames),50,50))



for module in tqdm(np.arange(1,25)):
    for output in np.arange(1,5):
        fnames=glob('{}Module_{}/{}/*.mat'.format(psfdir,module,module+0.1*output))
        for l in (xrange(len(fnames))):
            h=sio.loadmat(fnames[0])
            psf=np.zeros((len(fnames),np.shape(h['psf'])[0],np.shape(h['psf'])[1]))
            res=h['grid_res'][0][0]
            lam=[]
            for i,f in enumerate(fnames):
                h=sio.loadmat(f)
                psf[i,:,:]=h['psf']
                lam.append(f.split('_')[-1][0:-4])

            PRFx = np.arange(0., np.shape(psf[0])[0] + 0.)-np.shape(psf[0])[0]/2
            PRFy = np.arange(0., np.shape(psf[0])[1] + 0.)-np.shape(psf[0])[0]/2
            PRFx*=0.03
            PRFy*=0.03
            PRFX,PRFY=np.meshgrid(PRFx,PRFy)    
            cx=PRFX[0][np.where(PRFX[0]>=0)[0][0]:np.where(PRFX[0]<1)[0][-1]]
            cy=PRFY[:,0][np.where(PRFY[:,0]>=0)[0][0]:np.where(PRFY[:,0]<1)[0][-1]]
            cx,cy=np.meshgrid(cx,cy)
            prf=np.copy(cx)*0.+1.
            psf_c=convolve2d(psf[l,:,:],prf,mode='same')

            psf_c/=np.nansum(psf_c)

            model=RectBivariateSpline(PRFx,PRFy,psf_c)
            X,Y=np.meshgrid(np.linspace(-3,3,50),np.linspace(-3,3,50))

            PRFX,PRFY,psf_c=X,Y,model(X[0],Y[:,0])

            psf_model[module-1,output-1,l,:,:]=psf_c
pickle.dump(psf_model,open('psf_model.p','wb'))




  0%|          | 0/24 [00:00<?, ?it/s]




  8%|▊         | 2/24 [01:52<20:33, 56.08s/it]


 12%|█▎        | 3/24 [03:47<25:52, 73.94s/it]


 17%|█▋        | 4/24 [05:40<28:33, 85.70s/it]


 25%|██▌       | 6/24 [07:35<23:09, 77.21s/it]


 29%|██▉       | 7/24 [09:27<24:50, 87.66s/it]

In [ ]:
psf_model=pickle.load(open('psf_model.p','rb'))

In [ ]:
X,Y=np.meshgrid(np.linspace(-5,5,50),np.linspace(-5,5,50))
fnames=glob('{}Module_{}/{}/*.mat'.format(psfdir,2,2+1*0.1))
lam=np.asarray([(f.split('_')[-1])[0:-4] for f in fnames],dtype='float')

fnames=glob('../data/tabby/*.fits')


for f in fnames:
    h=fits.open(f)
    output=h[0].header['OUTPUT']
    module=h[0].header['MODULE']
    plt.figure()
    plt.imshow(psf_model[module-1,output-1,11,:,:])
    
    fit=fitting.LevMarLSQFitter()
    

    psf_xshifts=np.zeros(np.shape(psf_model)[1])
    psf_yshifts=np.zeros(np.shape(psf_model)[1])

    for l in (xrange(len(fnames))):
        g_init=models.Gaussian2D(x_mean=0,y_mean=0,amplitude=np.max(psf_model[module-1,output-1,l,:,:]))
        g=fit(g_init,X,Y,psf_model[l,:,:])
        psf_xshifts[l],psf_yshifts[l]=g.x_mean.value,g.y_mean.value
    plt.figure()
    plt.plot(psf_xshifts)
    plt.plot(psf_yshifts)
    break
    